# Assignment 5 - Text Analysis
An explanation this assignment could be found in the .pdf explanation document


## Materials to review for this assignment
<h4>From Moodle:</h4> 
<h5><u>Review the notebooks regarding the following python topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Working with strings</b> (tutorial notebook)<br/>
&#x2714; <b>Text Analysis</b> (tutorial notebook)<br/>
&#x2714; <b>Hebrew text analysis tools (tokenizer, wordnet)</b> (moodle example)<br/>
&#x2714; <b>(brief review) All previous notebooks</b><br/>
</div> 
<h5><u>Review the presentations regarding the following topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Text Analysis</b> (lecture presentation)<br/>
&#x2714; <b>(brief review) All other presentations</b><br/>
</div>

## Preceding Step - import modules (packages)
This step is necessary in order to use external modules (packages). <br/>

In [58]:
# --------------------------------------
import pandas as pd
import numpy as np
# --------------------------------------


# --------------------------------------
# ------------- visualizations:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
# --------------------------------------


# ---------------------------------------
import sklearn
from sklearn import preprocessing, metrics, pipeline, model_selection, feature_extraction 
from sklearn import naive_bayes, linear_model, svm, neural_network, neighbors, tree
from sklearn import decomposition, cluster

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# ---------------------------------------


# ----------------- output and visualizations: 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# ---------------------------------------

### Text analysis and String manipulation imports:

In [59]:
# --------------------------------------
# --------- Text analysis and Hebrew text analysis imports:
# vectorizers:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# regular expressions:
import re
# --------------------------------------

### (optional) Hebrew text analysis - WordNet (for Hebrew)
Note: the WordNet is not a must

#### (optional) Only if you didn't install Wordnet (for Hebrew) use:

In [60]:
# word net installation:

# unmark if you want to use and need to install
!pip install wn

In [61]:
# word net import:
import wn

### (optional) Hebrew text analysis - hebrew_tokenizer (Tokenizer for Hebrew)
Note: the hebrew_tokenizer is not a must

#### (optional) Only if you didn't install hebrew_tokenizer use:

In [62]:
!pip install hebrew_tokenizer

In [63]:

import hebrew_tokenizer as ht

### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [64]:
train_filename = 'annotated_corpus_for_train.csv'
test_filename  = 'corpus_for_test.csv'
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')

In [65]:
df_train.head(8)
df_train.shape

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",m
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,f
5,לפני כ3 חודשים טסתי לרומא למשך שבוע. טסתי במטו...,f
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,m
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב. הדירה ה...,f


(753, 2)

In [66]:
df_test.head(3)
df_test.shape

,test_example_id,story
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...


(323, 2)

### Your implementation:
Write your code solution in the following code-cells

In [67]:
#Preprocessing

In [68]:
# tokenize the stories by words and replace them with stories that contain only words
stories = []
for story in df_train['story']:
    tokens = ht.tokenize(story)  
    tokenized = ''
    for grp, token, token_num, (start_index, end_index) in tokens:
        if grp =='HEBREW':
            tokenized += token + ' '
    stories.append(tokenized)
df_train['story'] = stories

In [69]:
df_train['gender'] = np.where(df_train['gender'] == 'm',1,0)

In [70]:
x_train = df_train['story']
y_train = df_train['gender']

In [71]:
# training + cross validation

In [72]:
# tune the best hyperparameters for learning
tfidf_vec = TfidfVectorizer() # counts every token and evaluate it importance
classifier = SGDClassifier(max_iter = 1000,random_state=42)
line = Pipeline([('tfidf',tfidf_vec),('clf',classifier)])#combines these two
params = {'tfidf__max_df':[0.5,0.7,0.9],#evaluating it multiple times with 50% more, 70%, 90%
          'tfidf__max_features':[150,200,250,300,350,1000], #most common words
          'clf__loss':['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'], #calc error rate and determine optimal function
          'clf__penalty':['l2', 'l1', 'elasticnet'], #regulation to prevent overfitting
          'clf__alpha':[0.0001,0.092,0.001]}#control regulation
grid_search = GridSearchCV(line, cv=10, param_grid=params,scoring='f1_macro')#line = type of machine learning pipline 
#cv = 10 devide it to 10 folds
#param_grid the hyperparameters above
#scoring = picks the best score from the 10 folds
best_clf = grid_search.fit(x_train,y_train)
print("Best parameters from gridsearch: {}".format(best_clf.best_params_))
print("CV score=%0.3f" % best_clf.best_score_)
gender_clf = best_clf.best_estimator_

Best parameters from gridsearch: {'clf__alpha': 0.0001, 'clf__loss': 'hinge', 'clf__penalty': 'elasticnet', 'tfidf__max_df': 0.9, 'tfidf__max_features': 1000}
CV score=0.677


In [73]:
#Training score

In [74]:
y_pred = gender_clf.predict(x_train)# Comparing the training data with the prediction
pre_macro = metrics.precision_score(y_train, y_pred, average="macro")
recall_macro = metrics.recall_score(y_train, y_pred, average="macro")
f1_macro = 2 * pre_macro * recall_macro / (pre_macro + recall_macro )
print("training score=%0.4f" % f1_macro)
pd.DataFrame({'y_train':y_train, 'y_pred':y_pred}).head(10)

training score=0.9963


,y_train,y_pred
0,1,1
1,1,1
2,0,0
3,1,1
4,0,0
5,0,0
6,1,1
7,0,0
8,1,1
9,0,0


In [75]:
#Test prediction:

In [77]:
# repeating the token proccess on the test data
t_stories = []
for story in df_test['story']:
    tokens = ht.tokenize(story)  
    tokenized = ''
    for grp, token, token_num, (start_index, end_index) in tokens:
        if grp =='HEBREW':
            tokenized += token + ' '
    t_stories.append(tokenized)
df_test['story'] = t_stories           
x_test = df_test['story']
y_predicted = gender_clf.predict(x_test)

In [78]:
# convert the results from binary values to categorial values
y_predicted = np.where(y_predicted == 1,'m','f')

In [79]:
# create dataframe with the test results
df_predicted = pd.DataFrame({'test_example_id':df_test['test_example_id'],'story':df_test['story'],'predicted_gender':y_predicted})
df_predicted

,test_example_id,story,predicted_gender
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...,m
1,1,הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת י...,m
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...,m
3,3,רגע הגיוס לצבא היה הרגע הכי משמעותי עבורי אני ...,f
4,4,אני הגעתי לברזיל ישר מקולומביה וגם אני עשיתי ע...,f
...,...,...,...
318,318,בשנה האחרונה הרגשתי די תקוע בעבודה השגרה הפכה ...,m
319,319,אני ואילן חברים טובים מזה שנה תמיד חלמנו לפתח ...,m
320,320,מידי יום שישי אני נוהג לנסוע בתחבורה ציבורית ס...,f
321,321,לפני מספר חודשים בשיא התחלואה של הגל השני עמית...,m


### Save output to csv (optional)
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following:

In [ ]:
df_predicted.to_csv('classification_results.csv',index=False)